In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [2]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_2149/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [4]:
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [7]:
all_data[['work_age', 'workYear']]

,work_age,workYear
0,5-10年,13.0
1,3-5年,7.0
2,10年以上,14.0
3,3-5年,3.0
4,3-5年,5.0
...,...,...
81291,5-10年,1.0
81292,10年以上,23.0
81293,10年以上,23.0
81294,3-5年,5.0


## 理清需要用的特征


In [ ]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 'pay_months', 'currentSalary', 'desiredSalary']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description', 'requirement', 'work_duration_mean', 'equal_words', 'equal_job']


wide_columns = []
deep_columns = []